In [1]:
from src.trading import TradingEnv
env = TradingEnv(
    dataset_path='./data/eurusd_minute.csv',
    spread=0.0002,
    period=100, sold=1000,
    min_sold=100, nlot=0.1,
    episode_lenght=1)

In [2]:
from keras.layers import Dense, Activation
from keras.models import Sequential, load_model
from keras.optimizers import Adam
import numpy as np
import wandb

class ReplayBuffer(object):
    def __init__(self, max_size, input_shape, n_actions, discrete=False):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.discrete = discrete
        self.state_memory = np.zeros((self.mem_size, input_shape))
        self.new_state_memory = np.zeros((self.mem_size, input_shape))
        dtype = np.int8 if self.discrete else np.float32
        self.action_memory = np.zeros((self.mem_size, n_actions), dtype=dtype)
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.float32)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        # store one hot encoding of actions, if appropriate
        if self.discrete:
            actions = np.zeros(self.action_memory.shape[1])
            actions[action] = 1.0
            self.action_memory[index] = actions
        else:
            self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = 1 - done
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size)

        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]

        return states, actions, rewards, states_, terminal

def build_dqn(lr, n_actions, input_dims, fc1_dims, fc2_dims):
    model = Sequential([
                Dense(fc1_dims, input_shape=(input_dims,)),
                Activation('relu'),
                Dense(fc2_dims),
                Activation('relu'),
                Dense(n_actions)])

    model.compile(optimizer=Adam(lr=lr), loss='mse')

    return model

class DDQNAgent(object):
    def __init__(self, alpha=0.01, gamma=0.99, n_actions=4, epsilon=1.0, batch_size=30,
                 input_dims=403, epsilon_dec=0.996,  epsilon_end=0.01,
                 mem_size=1000000, fname='ddqn_model.h5', replace_target=100):
        self.action_space = [i for i in range(n_actions)]
        self.n_actions = n_actions
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_dec = epsilon_dec
        self.epsilon_min = epsilon_end
        self.batch_size = batch_size
        self.model_file = fname
        self.replace_target = replace_target
        self.memory = ReplayBuffer(mem_size, input_dims, n_actions,
                                   discrete=True)
        self.q_eval = build_dqn(alpha, n_actions, input_dims, 1618, 1618)
        self.q_target = build_dqn(alpha, n_actions, input_dims, 1618, 1618)

    def remember(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)

    def choose_action(self, state):
        state = state[np.newaxis, :]
        rand = np.random.random()
        if rand < self.epsilon:
            action = np.random.choice(self.action_space)
        else:
            actions = self.q_eval.predict(state)
            action = np.argmax(actions)

        return action

    def learn(self):
        if self.memory.mem_cntr > self.batch_size:
            state, action, reward, new_state, done = \
                                          self.memory.sample_buffer(self.batch_size)

            action_values = np.array(self.action_space, dtype=np.int8)
            action_indices = np.dot(action, action_values)

            q_next = self.q_target.predict(new_state)
            q_eval = self.q_eval.predict(new_state)
            q_pred = self.q_eval.predict(state)

            max_actions = np.argmax(q_eval, axis=1)

            q_target = q_pred

            batch_index = np.arange(self.batch_size, dtype=np.int32)

            q_target[batch_index, action_indices] = reward + \
                    self.gamma*q_next[batch_index, max_actions.astype(int)]*done

            _ = self.q_eval.fit(state, q_target, verbose=0)

            self.epsilon = self.epsilon*self.epsilon_dec if self.epsilon > \
                           self.epsilon_min else self.epsilon_min
            if self.memory.mem_cntr % self.replace_target == 0:
                self.update_network_parameters()

    def update_network_parameters(self):
        self.q_target.set_weights(self.q_eval.get_weights())

    def save_model(self):
        self.q_eval.save(self.model_file)

    def load_model(self):
        self.q_eval = load_model(self.model_file)
        # if we are in evaluation mode we want to use the best weights for
        # q_target
        if self.epsilon == 0.0:
            self.update_network_parameters()

In [ ]:
BUY = 0
SELL = 1
CLOSE = 2
HOLD = 3
agentoo7 = DDQNAgent()
run = wandb.init(project="DeepTrading", entity="smonnier")
num_episodes = 5000
max_step = 1440
solde = 1000
benefice = 0
total_reward = 0
try:
    for episode in range(num_episodes):
        print(
            "Training episode: {0}/{1}".format(episode+1, num_episodes), end="\r")

        state = env.reset()
        episode_reward = 0
        episode_action = [0,0,0,0]
        for step in range(max_step):

            action = agentoo7.choose_action(state)
            episode_action[action] += 1
            new_state, reward, done = env.step(action)
            agentoo7.remember(state, action, reward, new_state, done)
            agentoo7.learn()
            state = new_state
            total_reward += reward
            episode_reward += reward


            if done:
                break

        benefice = benefice + env.sold - solde

        

        wandb.log({"reward": episode_reward, "duration": step,
                       "solde": env.sold, "total_benefice": benefice, 
                       "Buy": episode_action[BUY],
                       "Sell": episode_action[SELL],
                       "Hold": episode_action[HOLD],
                       "Close": episode_action[CLOSE],
                       "Total trade": env.total_trade,
                       "Total reward":total_reward})
    run.finish()
    agentoo7.save_model()  # save trained model
except KeyboardInterrupt:
    agentoo7.save_model()
    run.finish()
    print("Training has been interrupted")

C:\Users\smonn\miniconda3\envs\deeptrading\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
wandb: Currently logged in as: smonnier (use `wandb login --relogin` to force relogin)
